# 구내식당 식수 인원 예측 AI 경진대회
- 대회 기간 : 2021.06.03 ~ 2021.07.23
- `public score : 85.9608`, `private score : 123.468`
- 사용 모델 : Pycaret을 활용한 Auto ML 모델
- 대회 task : 정형데이터 - 회귀

In [ ]:
from pycaret.regression import *
import datetime as dt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
!pip install catboost
!pip install pycaret

In [26]:
def preprocessing(df, mode):
    """
    데이터를 전처리하는 함수
    """

    if mode == 'trian':
        df[['현본사소속재택근무자수', '중식계', '석식계']] = df[[
            '현본사소속재택근무자수', '중식계', '석식계']].astype(int)
    else:
        df['현본사소속재택근무자수'] = df['현본사소속재택근무자수'].astype('int')

    df['일자'] = pd.to_datetime(df['일자'])

    df['년'] = df['일자'].dt.year
    df['월'] = df['일자'].dt.month
    df['일'] = df['일자'].dt.day
    df['주'] = df['일자'].dt.week
    df['요일'] = df['일자'].dt.weekday
    df['출근'] = df['본사정원수']-(df['본사휴가자수']+df['본사출장자수']+df['현본사소속재택근무자수'])
    df['휴가비율'] = df['본사휴가자수']/df['본사정원수']
    df['출장비율'] = df['본사출장자수']/df['본사정원수']
    df['야근비율'] = df['본사시간외근무명령서승인건수']/df['출근']
    df['재택비율'] = df['현본사소속재택근무자수']/df['본사정원수']
    df.drop(['일자', '조식메뉴', '중식메뉴', '석식메뉴'], axis=1, inplace=True)

    print(mode, 'data preprocessing done')
    return df

In [28]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train = preprocessing(train, 'train')
test = preprocessing(test, 'test')

train data preprocessing done
test data preprocessing done


In [39]:
train_lunch = train.drop('석식계', axis=1)
train_dinner = train.drop('중식계', axis=1)

In [45]:
def lunch_predict():
    """
    pycaret을 이용해 중식계를 예측
    """
    reg = setup(data=train_lunch,
                target='중식계',
                numeric_imputation='mean',
                normalize=True,
                silent=True)
    # 성능 좋은 모델 중 5개를 앙상블 하여 사용
    best_5_model = compare_models(sort='MAE', n_select=5)
    # compare_models를 이용해 뽑은 5개의 모델을 혼합하여 모델을 생성합니다.
    # 혼합시 사용할 fold의 수와 optimizer 종류를 입력해 줍니다.
    blended_model = blend_models(
        estimator_list=best_5_model, fold=10, optimize='MAE')
    # 생성한 모델을 최종적으로 사용할 모델로 변환
    # 이 과정에서는 train_size = 1로, 모든 데이터를 사용해 재학습합니다
    final_model = finalize_model(blended_model)
    test_pred = predict_model(final_model, test)
    return test_pred

In [46]:
def dinner_predict():
    """
    pycaret을 이용해 석식계를 예측
    """

    reg = setup(data=train_dinner,
                target='석식계',
                numeric_imputation='mean',
                normalize=True,
                silent=True)
    # 성능 좋은 모델 중 5개를 앙상블 하여 사용
    best_5_model = compare_models(sort='MAE', n_select=5)
    # compare_models를 이용해 뽑은 5개의 모델을 혼합하여 모델을 생성합니다.
    # 혼합시 사용할 fold의 수와 optimizer 종류를 입력해 줍니다.
    blended_model = blend_models(
        estimator_list=best_5_model, fold=10, optimize='MAE')
    # 생성한 모델을 최종적으로 사용할 모델로 변환
    # 이 과정에서는 train_size = 1로, 모든 데이터를 사용해 재학습합니다
    final_model = finalize_model(blended_model)
    test_pred = predict_model(final_model, test)
    return test_pred

In [47]:
lunch_predict = lunch_predict()
dinner_predict = dinner_predict()

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,45.6691,3612.6890,60.1056,0.7923,0.9429,0.0974
1,51.8842,5889.0479,76.7401,0.5772,0.5872,0.1211
2,42.4161,3299.7549,57.4435,0.7430,0.4575,0.0995
3,59.4542,9181.2499,95.8188,0.6242,1.3735,0.1069
4,55.1006,8504.3662,92.2191,0.5806,1.0843,0.1236
5,55.1124,6798.4859,82.4529,0.6500,0.8905,0.1118
6,49.5440,4440.2457,66.6352,0.7942,0.9486,0.1111
7,55.6776,6301.1364,79.3797,0.6072,0.8754,0.1166
8,51.3022,5436.2673,73.7310,0.6489,0.5604,0.1166
9,47.5441,6425.7983,80.1611,0.5432,0.8705,0.0938


In [49]:
submission = pd.read_csv('sample_submission.csv')
submission['중식계'] = lunch_predict['Label']
submission['석식계'] = dinner_predict['Label']
submission.head()

,일자,중식계,석식계
0,2021-01-27,982.442145,202.899999
1,2021-01-28,925.973782,412.301450
2,2021-01-29,577.873855,257.715289
3,2021-02-01,1246.089715,528.612609
4,2021-02-02,980.553761,478.438495


In [50]:
submission.to_csv('hyup.csv',index=False)